# Configuración (importar dependencias, librerías, ...)

In [ ]:
# Set the seed value all over the place to make this reproducible.
# esto hay que ponerlo justo antes de importar para que los experimentos
# sean reproducible
!pip install sentencepiece
!pip install pytorch-lightning
!pip install emoji
import random
import torch
import numpy as np
import os
from pytorch_lightning import seed_everything

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)# Store the average loss after eachepoch so we can plot them.
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ["TF_DETERMINISTIC_OPS"] = "1" # See:https://github.com/NVIDIA/tensorflow-determinism#confirmed-current-gpu-specific-sources-of-non-determinism-with-solutions
seed_everything(42, workers=True)

!pip install transformers datasets
#!pip install textblob
from google.colab import drive
from datasets import Dataset, DatasetDict, load_metric
import pandas as pd
import sklearn as sk
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
 TrainingArguments, Trainer, pipeline, EarlyStoppingCallback


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.6/715.6 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel fo

INFO:lightning_fabric.utilities.seed:Global seed set to 42


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.0 MB/s eta 0:00:00


In [ ]:
# Check that pyTorch is identifying the GPU
if torch.cuda.device_count() > 0:
  print(f'GPU detected. Currently using: "{torch.cuda.get_device_name(0)}"')
else:
  print('Currently using CPU, change the type of the runtime in the \'runtime\' tab')

GPU detected. Currently using: "Tesla T4"


# Preparación de los datos

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Funciones de limpieza
import re

# Función de eliminación de contracción
def expand_contraction(tweet):
    tweet = contractions.fix(tweet)
    return tweet

# Función para corregir los typos
def correct_spelling(tweet):
    tweetBlob = TextBlob(tweet)
    tweet = tweetBlob.correct()
    return tweet
def remove_links(tweet):
    """Takes a string and removes web links from it"""
    tweet = re.sub(r'http\S+', '', tweet)   # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # remove bitly links
    tweet = re.sub(r'\[link\]', '', tweet)   # remove [links]
    tweet = re.sub(r'pic.twitter\S+','', tweet)
    return tweet

def remove_users(tweet):
    """Takes a string and removes retweet and @user information"""
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove tweeted at
    return tweet

def remove_hashtags(tweet):
    """Takes a string and removes any hash tags"""
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove hash tags
    return tweet

def remove_av(tweet):
    """Takes a string and removes AUDIO/VIDEO tags or labels"""
    tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    return tweet
# Esto solo es para ver la longitud (en palabras) de los tweets
def divide(texto):
  return texto.split()

def cuenta_tokens(lista):
  return len(lista)

#!pip install emoji
#import emoji
#def transforma_emoji(tweet):
#
#    tweet = emoji.demojize(tweet)
#
#    return tweet

## Lectura de los ficheros

In [ ]:
# usar cuando tenemos ficheros de train y test separados
train_data_path = '/content/drive/MyDrive/DueloSano/Nuevos-EN-2023/New_EN_DS_Entrenamiento.tsv'
test_data_path = '/content/drive/MyDrive/DueloSano/Nuevos-EN-2023/New_EN_DS_Test.tsv' ### este no vale para nada
aumentado_path = '/content/drive/MyDrive/DueloSano/Nuevos-EN-2023/New_EN_DS_EntrenamientoAumentado.tsv'
# Lo pasamos a un dataframe
train_df_full = pd.read_csv(train_data_path, encoding = 'UTF-8', sep='\t')
train_df_aumentado = pd.read_csv(aumentado_path, encoding = 'UTF-8', sep='\t')
print("distribucion del fichero de aumento: ",train_df_aumentado.value_counts("label"))
#entrenar con train test y validacion con split
print("distribucion original - train: ",train_df_full.value_counts("label"))
train_df_full = train_df_aumentado # quitar cuando no queramos usar fichero aumentado
#df_0 = train_df_full[train_df_full["label"]==0][0:3000]
df_0 = train_df_full[train_df_full["label"]==0]#.sample(n=484,random_state=42)
df_1 = train_df_full[train_df_full["label"]==1][:]

train_df_under = pd.concat([df_0,df_1]) # se llama under pero vale para todos
print("distribucion despues del Xsampling TRAIN: ",train_df_under.value_counts("label"))


train_df, valid_df = train_test_split(train_df_under, test_size = 0.2, shuffle = True, stratify=train_df_under[['label']])
valid_df_0 = valid_df[valid_df["label"]==0]#.sample(n=121,random_state=42)
valid_df_1 = valid_df[valid_df["label"]==1]#.sample(n=50,random_state=42)
valid_df = pd.concat([valid_df_0,valid_df_1])
print("distribucion final TRAIN: ",train_df.value_counts("label"))
print("distribucion final validacion: ",valid_df.value_counts("label"))
test_df_full = pd.read_csv(test_data_path, encoding = 'UTF-8', sep='\t')
print("distribucion original - test: ",test_df_full.value_counts("label"))

dft_0 = test_df_full[test_df_full["label"]==0]#[:100] #123
dft_1 = test_df_full[test_df_full["label"]==1]#[:100]
test_df = pd.concat([dft_0,dft_1])
print("distribucion despues del undersampling test: ",test_df.value_counts("label"))
print("Ejemplos del conjunto completo: ", len(train_df_full))
print("Ejemplos usados para entrenar: ", len(train_df))
print("Ejemplos usados para validar: ", len(valid_df))
print("Ejemplos usados para test: ", len(test_df))

train_df = train_df.rename(columns={'tweet':'text'})
valid_df = valid_df.rename(columns={'tweet':'text'})
test_df = test_df.rename(columns={'tweet':'text'})
t0 = train_df[train_df["label"]==0]
t1 = train_df[train_df["label"]==1]
# Calculate weights manually
print(f'The weights for the majority class is {1/(len(t0)/(len(t0) + len(t1))):.3f}')
print(f'The weights for the minority class is {1/(len(t1)/(len(t0) + len(t1))):.3f}')
c0=1/(len(t0)/(len(t0) + len(t1)))
c1=1/(len(t1)/(len(t0) + len(t1)))


distribucion del fichero de aumento:  label
0    1116
1     968
dtype: int64
distribucion original - train:  label
0    1116
1     484
dtype: int64
distribucion despues del Xsampling TRAIN:  label
0    1116
1     968
dtype: int64
distribucion final TRAIN:  label
0    893
1    774
dtype: int64
distribucion final validacion:  label
0    223
1    194
dtype: int64
distribucion original - test:  label
0    279
1    121
dtype: int64
distribucion despues del undersampling test:  label
0    279
1    121
dtype: int64
Ejemplos del conjunto completo:  2084
Ejemplos usados para entrenar:  1667
Ejemplos usados para validar:  417
Ejemplos usados para test:  400
The weights for the majority class is 1.867
The weights for the minority class is 2.154


In [ ]:
train_df

,id,text,label
2013,1255648913702477820,"#COVID19 If you feel sad and hopeless, talk to...",1
102,1245354314069131266,4/ Hessen Finance Minister commits suicide in ...,1
1041,1255629043237601280,@USER what will be your support for older adul...,0
1044,1246963356663255040,Death of ‘Menudo’ by coronavirus turns on alar...,0
224,1254344581300314110,on @user: will the Coronavirus change our atti...,1
...,...,...,...
420,1246304340291268610,the heartbreaking testimony (english) of a wom...,1
1899,1255698990621958144,How tremendous... Younger victim of the corona...,1
602,1251541971010805760,"On March 23, the first case of Covid-19 was co...",0
2065,1247339592925220860,To take into account that the stress of the si...,1


## Limpieza de datos

In [ ]:
train_df['text'] =train_df['text'].apply(remove_av)
train_df['text'] =train_df['text'].apply(remove_links)
#train_df['text'] =train_df['text'].apply(remove_users)
#train_df['text'] = train_df['text'].apply(transforma_emoji)

test_df['text']= test_df['text'].apply(remove_av)
test_df['text']= test_df['text'].apply(remove_links)
#test_df['text']= test_df['text'].apply(remove_users)
#test_df['text'] = test_df['text'].apply(transforma_emoji)
valid_df['text']=valid_df['text'].apply(remove_av)
valid_df['text']=valid_df['text'].apply(remove_links)
#valid_df['text']=valid_df['text'].apply(remove_users)
#valid_df['text'] = valid_df['text'].apply(transforma_emoji)

train_df['text'] = train_df['text'].str.lower()
test_df['text'] = test_df['text'].str.lower()
valid_df['text'] = valid_df['text'].str.lower()
train_df
train_df


,id,text,label
2013,1255648913702477820,"#covid19 if you feel sad and hopeless, talk to...",1
102,1245354314069131266,4/ hessen finance minister commits suicide in ...,1
1041,1255629043237601280,@user what will be your support for older adul...,0
1044,1246963356663255040,death of ‘menudo’ by coronavirus turns on alar...,0
224,1254344581300314110,on @user: will the coronavirus change our atti...,1
...,...,...,...
420,1246304340291268610,the heartbreaking testimony (english) of a wom...,1
1899,1255698990621958144,how tremendous... younger victim of the corona...,1
602,1251541971010805760,"on march 23, the first case of covid-19 was co...",0
2065,1247339592925220860,to take into account that the stress of the si...,1


In [ ]:
# Convertimos los dataframes en objetos datasets
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

#print(train_dataset)
#print(train_dataset, valid_dataset)


In [ ]:
# Mostramos los datos en formato pandas
train_dataset.set_format("pandas")
train_dataset[:]

,id,text,label,__index_level_0__
0,1255648913702477820,"#covid19 if you feel sad and hopeless, talk to...",1,2013
1,1245354314069131266,4/ hessen finance minister commits suicide in ...,1,102
2,1255629043237601280,@user what will be your support for older adul...,0,1041
3,1246963356663255040,death of ‘menudo’ by coronavirus turns on alar...,0,1044
4,1254344581300314110,on @user: will the coronavirus change our atti...,1,224
...,...,...,...,...
1662,1246304340291268610,the heartbreaking testimony (english) of a wom...,1,420
1663,1255698990621958144,how tremendous... younger victim of the corona...,1,1899
1664,1251541971010805760,"on march 23, the first case of covid-19 was co...",0,602
1665,1247339592925220860,to take into account that the stress of the si...,1,2065


In [ ]:
# Reseteamos el formato para que no haya fallos
train_dataset.reset_format()
valid_dataset.reset_format()

# Borramos los dataframes puesto que no los vamos a usar más
#del train_df_palabras
#del train_df_full
#del train_df
#del test_df

# Preparación de los conjuntos para el entrenamiento

In [ ]:
# Asignamos una etiqueta numérica en función de la etiqueta principal
#   Label = 0 --> negativo
#   Label = 1 --> positivo
# Para EDOS es 'label_sexist'
def set_labels(records):
  if records['label'] == 0:
    label = 0
  else:
    label = 1
  return {'labels': label}

In [ ]:
# Correct the labels of the test split and create a Dataset dict for all the splits (except test_dataset)
# Esto es para cuando tengamos fichero de train y de valid
# dataset = DatasetDict({'train': train_dataset, 'valid': valid_dataset})
dataset_train = train_dataset
dataset_valid = valid_dataset
#dataset = dataset.remove_columns('__index_level_0__')

# Map the functions to the dataset
dataset_train = dataset_train.map(set_labels)
dataset_valid = dataset_valid.map(set_labels)

print(dataset_train, dataset_valid)

Map:   0%|          | 0/1667 [00:00<?, ? examples/s]

Map:   0%|          | 0/417 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'text', 'label', '__index_level_0__', 'labels'],
    num_rows: 1667
}) Dataset({
    features: ['id', 'text', 'label', '__index_level_0__', 'labels'],
    num_rows: 417
})


In [ ]:
# Reseteamos el formato para que no haya fallos
dataset_train.reset_format()
dataset_valid.reset_format()

# Proceso de clasificación

## Tokenización

In [ ]:
columns_train = dataset_train.column_names  # Coge todas las columnas
columns_valid = dataset_valid.column_names  # Coge todas las columnas
columns_train.remove("labels") # Elimina la columna "labels"
columns_valid.remove("labels") # Elimina la columna "labels"


## Definición de la métricas

In [ ]:
# Función para realizar distintas métricas en ejecución

def compute_metrics(eval_pred):
  """
  Compute metrics for Trainer
  """

  labels = eval_pred.label_ids
  preds = eval_pred.predictions.argmax(-1)

  #preds = np.argmax(preds, axis=-1)
  precision, recall, f1, _ = sk.metrics.precision_recall_fscore_support(labels, preds, average="macro")
  f1_minoritaria= f1_score(labels, preds,pos_label=1)
  f1_mayoritaria = f1_score(labels,preds,pos_label=0)
  acc = sk.metrics.accuracy_score(labels, preds)
  AUC = roc_auc_score(labels, preds)
  PREC_REC = average_precision_score(labels, preds)
  return { 'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall, 'AUC': AUC ,
           'f1_minoritaria': f1_minoritaria, 'f1_mayoritaria': f1_mayoritaria, 'PREC_REC': PREC_REC}


## WandDB


In [ ]:
# para usar wandb
!pip install wandb
#WANDB_PROJECT="nombre proyecto"
import wandb
wandb.login()

n_labels = 2
epochs = 9
sweep_config = {
    "name": "Ingles- OVERsampling - Duelo Sano microsoft/deberta-base",
    'description': 'ingles OVERsampling microsoft/deberta-v3-base',
    'method': 'grid', #grid, random
    'metric': {
      'name': 'eval/AUC',
      'goal': 'maximize'
    },
    'parameters': {
      #  'epochs':{
      #      'values':[2, 3, 4]
      #  },
        'model': {'values':[
                        #    'microsoft/deberta-v3-base',
                            "microsoft/deberta-base"#,
                        #    'bert-base-uncased',
                        #    'roberta-base'
                            ]},
        'weight_decay':{'values':[0.1,0.01,0.001]},

        'max_len' :{'values': [64,128,256]},
        'learning_rate': {
            'values': [ 3e-5, 2e-5, 5e-5]
        },
        'batch_size': {
            'values': [32,64,16]
        }

    }
}

#esta linea hay que quitarla cuando queramos continuar una sweep no exhausta
sweep_id = wandb.sweep(sweep_config,project = "DueloSanoFebrero2023")
# Se definen los parámetros del Trainer()



def train():
    #id = wandb.util.generate_id()
    id='1111'
    wandb.init(id=id, resume="allow")
    print("******************** nuevo barrido *********************************")
    print("********************************************************************")
    print("config ",wandb.config)
    #print(sweep_id)
    print("*******************************************************************")


    tokenizerWandb = AutoTokenizer.from_pretrained(wandb.config.model)
    def tokenize_dataWandb(examples):
               return tokenizerWandb(examples["text"], truncation=True, max_length=wandb.config.max_len, padding=True)
    encoded_dataset_train = dataset_train.map(tokenize_dataWandb, batched=True, remove_columns=columns_train)
    encoded_dataset_valid = dataset_valid.map(tokenize_dataWandb, batched=True, remove_columns=columns_valid)
    num_train_samples = int(len(encoded_dataset_train))
    num_evaluation= int(len(encoded_dataset_valid))
    logging_steps = max(1,len(encoded_dataset_train) // (2 * wandb.config.batch_size * epochs))
    optim=["adamw_hf", "adamw_torch", "adamw_apex_fused","adafactor","adamw_torch_xla"]
    print("********************** loggin_steps", logging_steps)
    print(num_train_samples)
    print(2 * wandb.config.batch_size * epochs)
    print("********************** len", len(encoded_dataset_train))
    training_args = TrainingArguments(
      output_dir = 'results',
      num_train_epochs = epochs,
      learning_rate = wandb.config.learning_rate,
      per_device_train_batch_size = wandb.config.batch_size,
      per_device_eval_batch_size = wandb.config.batch_size,
      load_best_model_at_end = True,
      metric_for_best_model = 'AUC',
      #metric_for_best_model = 'f1-score',
      #metric_for_best_model = 'eval_loss',
      weight_decay = wandb.config.weight_decay,
      evaluation_strategy = 'epoch',
      save_strategy = 'epoch',
      logging_steps = logging_steps,
      save_total_limit = 3,
      optim = optim[1],
      push_to_hub=False,
      report_to = "wandb",
      run_name = "enero"
    #push_to_hub=True,
    #push_to_hub_model_id=f"{model_name}-finetuned-amazon_reviews_multi"
    )
    print("config ",wandb.config)
    #print("********************* SWEEP_ID ",sweep_id)
    def model_init():
        return AutoModelForSequenceClassification.from_pretrained(wandb.config.model,
                                                           num_labels=2,
                                                           output_attentions = False, # Whether the model returns attentions weights.
                                                           output_hidden_states = False,
                                                           return_dict=True
                                                           )
    trainer = Trainer(
      model_init=model_init,
      #model = AutoModelSequenceClassification.from_pretrained(model,num_labels=n_labels),
      args = training_args,
      compute_metrics = compute_metrics,
      callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
      train_dataset = encoded_dataset_train,
      #eval_dataset=dataset['valid'],
      eval_dataset = encoded_dataset_valid,
      tokenizer = tokenizerWandb
    )
    trainer.train()

    wandb.finish()





#hay que poner sweep_id cuando es la primera vez o el agent "i2c/DueloSanoFebrero2023/yz22vzbc" cuando
#queramos seguir ejecutando una sweep anterior
wandb.agent(sweep_id, function = train, count = 300)
#wandb.agent("i2c/DueloSanoFebrero2023/kfatcpri", function = train, count = 300)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 KB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 9.2 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=20df4ca8a271f468c772638bb2cd1f8365c1d05d4da6c1e3e749e972fba8ee3d
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:


Abort: ignored